In [16]:
import pandas as pd
from diskcache import Cache
from pathlib import Path
import requests
import json
import os
import re

Definition et creation des fonctions pour l'embeddings a partir de l'API JDM

Mettre les generiques + les infos semantiques (relation 36) + marqueurs de type (numero de la relation)

Pour le vecteur B regarder l'article (determine ou non determine)

FAIRE DES PRODUITS SCALAIRES SUR DES VECTEURS NORMES


Useful links :

https://www.jeuxdemots.org/jdm-about.php

https://jdm-api.demo.lirmm.fr/schema

https://jdm-api.demo.lirmm.fr/v0/relations/from/{node1_id}

https://jdm-api.demo.lirmm.fr/v0/relations/to/{node2_name}

https://jdm-api.demo.lirmm.fr/v0/node_by_name/{node_name}

In [7]:
CACHE_DIR = "./.jdm_cache"
relations_path = Path("/home/mario/Desktop/portfolio_projects/natural_language_semantics/relations")

In [8]:
cache = Cache()

In [33]:
def load_relations(directory_path: str):

    rows = []

    for filename in os.listdir(directory_path):
        full_path = os.path.join(directory_path, filename)
        relation_num = int(re.split("[-.]",filename)[0])
        if os.path.isfile(full_path):
            
            with open(full_path, "r", encoding="utf-8") as f:
                data = f.read()
                data = re.split("\n",data)
                
                for l in data:
                     rows.append({"content": l, "filename": relation_num})

           

    df = pd.DataFrame(rows)

    print(df.head())
    

    return df

In [10]:
def get_relations_JDM(node_name: str):
    with cache as c:
        if c.__contains__(node_name):
            return c.get(node_name)
        else:
            res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")
            res = json.loads(res.text)
            c.set(node_name,res)
            return res

In [34]:

df = load_relations(relations_path)

                     content  filename
0    Le travail de l’ouvrier         9
1   Le projet de l’ingénieur         9
2  Le discours du professeur         9
3     Le roman de l’écrivain         9
4    Le tableau de l’artiste         9


In [50]:
def strip_articles(s):
    if not s:
        return s
    s = s.replace("’", "'").strip()
    # articles + formes élidées
    return re.sub(r"^(le|la|les|un|une|l')\s", "", s, flags=re.IGNORECASE).strip()


In [51]:
df.iloc[:50,0].apply(lambda x: strip_articles(x))

0               travail de l'ouvrier
1              projet de l'ingénieur
2             discours du professeur
3                roman de l'écrivain
4               tableau de l'artiste
5                concert du musicien
6            découverte du chercheur
7                film du réalisateur
8               plan de l'architecte
9                     poème du poète
10             spectacle de l'acteur
11            rapport du journaliste
12          dessin de l'illustrateur
13      programme de l'informaticien
14           logiciel du développeur
15            peinture de la peintre
16                roman du romancier
17                 texte de l'auteur
18            chanson du compositeur
19    pièce de théâtre du dramaturge
20           manuscrit de l'écrivain
21              traité du philosophe
22              projet de l'étudiant
23          prototype de l'ingénieur
24              plan du constructeur
25                dessin du designer
26            sculpture de l'artiste
2

In [59]:
embedding = dict()
embedding["info_sem"] = []
embedding["type_marqueur"] = set()
embedding["hyperonyme"] = []

In [17]:
# obtenir les relations d'un mot
res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")

In [21]:
# parser le json
res = json.loads(res.text)

In [22]:
# res["nodes"]

[{'id': 25, 'name': 'décavaillonneuse', 'type': 1, 'w': 50},
 {'id': 60, 'name': 'bleu', 'type': 1, 'w': 7060},
 {'id': 93, 'name': 'saumurage', 'type': 1, 'w': 50},
 {'id': 176, 'name': 'fondations', 'type': 1, 'w': 449},
 {'id': 191, 'name': 'alluvionnage', 'type': 1, 'w': 50},
 {'id': 237, 'name': 'objet', 'type': 1, 'w': 8816},
 {'id': 267, 'name': 'Histoire', 'type': 1, 'w': 5653},
 {'id': 335, 'name': 'Virgile', 'type': 1, 'w': 90},
 {'id': 347, 'name': 'culinaire', 'type': 1, 'w': 279},
 {'id': 428, 'name': 'caveau', 'type': 1, 'w': 215},
 {'id': 441, 'name': 'kebab', 'type': 1, 'w': 93},
 {'id': 615, 'name': 'mazout', 'type': 1, 'w': 92},
 {'id': 626, 'name': 'peur', 'type': 1, 'w': 12557},
 {'id': 755, 'name': 'treille', 'type': 1, 'w': 89},
 {'id': 835, 'name': 'août', 'type': 1, 'w': 3560},
 {'id': 922, 'name': 'Pauillac', 'type': 1, 'w': 64},
 {'id': 967, 'name': 'date', 'type': 1, 'w': 6048},
 {'id': 1013, 'name': 'sacramentaire', 'type': 1, 'w': 51},
 {'id': 1086, 'name':

In [60]:
# filtrer les noeuds pour les relations voulues + les poids negatifs
for node in res["nodes"]:
    
    
    if(node["w"] <= 0 or node["type"] == 200): continue
    
    if(node["type"] == 36):
        embedding["info_sem"].append(node["id"])
    elif(node["type"] == 6):
        embedding["hyperonyme"].append(node["id"])
    
    embedding["type_marqueur"].add(node["type"])

In [61]:
embedding["hyperonyme"]

[151522,
 151550,
 151553,
 151564,
 151566,
 151567,
 151568,
 151576,
 151587,
 151663,
 151767,
 151926,
 152011,
 152012,
 152254,
 152775,
 152813,
 152836,
 152842,
 153037,
 153835,
 191679]

In [62]:
# utiliser le nombre de fois que la relation est presente comme poids ?
embedding["type_marqueur"]

{1, 2, 4, 5, 6, 8, 9, 10, 12, 18, 36, 444, 666, 777}

In [63]:
embedding["info_sem"]

[162759,
 162763,
 163012,
 163151,
 191741,
 217817,
 223172,
 223173,
 239128,
 241794,
 248198,
 251716,
 254876,
 254877,
 254878,
 264749,
 276096,
 317322,
 317324,
 317326,
 317329,
 1912917,
 1912988,
 2585996,
 2983124,
 15991432,
 50233540]